#sc‑RNA‑seq — Python (Scanpy)
- pipeline PBMC 3k analysis

# Install packeges 


In [ ]:
import scanpy as sc
adata = sc.datasets.pbmc3k()  # matrice gènes×cellules
adata

In [ ]:
import scanpy as sc
import numpy as np

# QC
adata.var['mt'] = adata.var_names.str.upper().str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, inplace=True)

# filter 
sc.pp.filter_cells(adata, min_counts=500)
sc.pp.filter_cells(adata, max_counts=40000)
adata = adata[adata.obs['pct_counts_mt'] < 10].copy()
sc.pp.filter_genes(adata, min_cells=3)

# Normalization 
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True, flavor='seurat')

# PCA
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')

# Voisinage, UMAP, clustering
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=1.0)

# Visualizations
sc.pl.umap(adata, color=['leiden', 'pct_counts_mt'], wspace=0.4)